# [主要的NLP任务](https://huggingface.co/learn/nlp-course/zh-CN/chapter7)
- Token 分类
- 掩码语言建模（如 BERT）
- 文本摘要
- 翻译
- 因果语言建模预训练（如 GPT-2）
- 问答

## Token 分类
目标是对文本中的每个 token（词或子词）进行分类。它是 序列标注（Sequence Labeling） 的核心任务，广泛应用于信息抽取、语法分析和语义理解等领域。常见的任务类型有：
- 命名实体识别（NER）：识别文本中的人名、地名、组织名等实体
- 词性标注（POS）：标注每个词的语法类别（名词、动词等）
- 分块（Chunking）：划分短语边界（如名词短语、动词短语）

In [ ]:
from datasets import load_dataset

# lhoestq/conll2003【其他数据集可能会下载不下来，所有换用这个数据集】
raw_datasets = load_dataset("lhoestq/conll2003", trust_remote_code=True)
print(raw_datasets)
# 查看训练集的第一个元素 和 NER 标签
print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])

# 查看 NER 标签的种类
ner_feature = raw_datasets["train"].features["ner_tags"]
print(ner_feature)
label_names = ner_feature.feature.names
print(label_names)

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

In [ ]:
# 创建 tokenizer 对象
from transformers import AutoTokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# is_split_into_words 用于指定输入文本是否已经预先分词（例如，按空格分割成单词）。
# 如果设置为 True，则分词器假定输入已经分割成单词，并对这些单词进行进一步的处理
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs)
print(inputs.tokens())
print("input tokens size:%d" % len(raw_datasets["train"][0]["tokens"]))
print("after tokenized size:%d" % len(inputs.tokens()))

In [ ]:

def align_labels_with_tokens(labels, word_ids):
    """
    将 NER 标签与分词后的 tokens 对齐
    args:
        labels (List[int]): NER 标签
        word_ids (List[int]): 分词后的 tokens
    return:
        new_labels (List[int]): 对齐后的标签
    """
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 新单词的开始!
            current_word = word_id
            # 将特殊的 token 设置为 -100
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # 特殊的token
            new_labels.append(-100)
        else:
            # 与前一个 tokens 类型相同的单词
            label = labels[word_id]
            # 如果标签是 B-XXX 我们将其更改为 I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels

labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(word_ids)
print(len(word_ids))
print(labels)
print(align_labels_with_tokens(labels, word_ids))

In [ ]:
def tokenize_and_align_labels(examples):
    """
    对数据集进行分词，并将 NER 标签与分词后的 tokens 对齐
    """
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs
# 对齐数据集中的所有数据
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

# step1 整理数据
# DataCollatorForTokenClassification 带有填充功能的Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
print(batch["labels"])
# 对比数据整理器的结果与数据集中未经处理的结果
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])
# step2 评估指标
import evaluate
metric = evaluate.load("seqeval")
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
print(labels)
# 更改索引 2 处的值，创建假的预测值，然后测试评估指标
predictions = labels.copy()
predictions[2] = "O"
print(metric.compute(predictions=[predictions], references=[labels]))
import numpy as np
#  自定义 compute_metrics() 函数，返回所需要的指标
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    # 删除忽略的索引(特殊 tokens )并转换为标签
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# step3 定义模型
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
print("num_labels=%d" % model.config.num_labels)

# step4 微调模型
from transformers import TrainingArguments, Trainer
# 设置 TrainingArguments
args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
# 自定义训练循环
from tqdm.auto import tqdm
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from accelerate import Accelerator
from transformers import get_scheduler

# 为数据集构建 DataLoader 
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

# 初始化模型
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

# 初始化优化器
optimizer = AdamW(model.parameters(), lr=2e-5)

# 将模型和优化器发送到 Accelerator，Accelerator 简化和优化分布式训练及硬件加速
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch
# 使用线性学习率调度器
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

model_name = "bert-finetuned-ner-accelerate"

# 计算 metric
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()
    # 删除忽略的索引(特殊 tokens )并转换为标签
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

# 初始化损失记录
train_losses = []
eval_losses = []

progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_train_epochs):
    # 训练
    model.train()
    total_train_loss = 0.0
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        total_train_loss += loss.item()  # 累加训练损失
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    # 计算平均训练损失
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    # 评估
    model.eval()
    total_eval_loss = 0.0
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss
            total_eval_loss += loss.item()  # 累加验证损失
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        # 填充模型的预测和标签后才能调用 gathere()
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)
        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)
    # 计算平均验证损失
    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    eval_losses.append(avg_eval_loss)
    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            "train_loss": avg_train_loss,
            "eval_loss": avg_eval_loss,
            "precision": results[f"overall_precision"],
            "recall": results[f"overall_recall"],
            "f1": results[f"overall_f1"],
            "accuracy": results[f"overall_accuracy"]
        },
    )
    # 保存模型
    output_dir = f"model/{model_name}-epoch{epoch}"
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        output_dir,
        is_main_process=accelerator.is_main_process,
        save_function=accelerator.save
    )
    # 同时保存训练状态
    accelerator.save(
        {"epoch": epoch, "optimizer_state": optimizer.state_dict(), "lr_scheduler_state": lr_scheduler.state_dict()}, 
        f"{output_dir}/training_state.pt"
    )
    if accelerator.is_main_process:
        print(f"\nModel saved to {output_dir}")

# """
# 保存最终的模型
# """
# # 全部训练完成后保存最终模型
# output_dir = f"{model_name}-final"
# accelerator.wait_for_everyone()
# unwrapped_model = accelerator.unwrap_model(model)
# unwrapped_model.save_pretrained(
#     output_dir,
#     is_main_process=accelerator.is_main_process,
#     save_function=accelerator.save
# )

In [ ]:
# 初始化加速器
accelerator = Accelerator()

# 1. 加载预训练的tokenizer和模型
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    "model/bert-finetuned-ner-accelerate-epoch2",
    id2label=id2label,
    label2id=label2id,
)

# 2. 加载测试集
test_dataloader = DataLoader(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    batch_size=8,
)

# 3. 准备测试集和模型
model, test_dataloader = accelerator.prepare(model, test_dataloader)

# 4. 预测测试集并计算指标
model.eval()
# 准备存储预测结果和真实标签
all_predictions = []
all_labels = []
# 迭代测试集
progress_test = tqdm(test_dataloader, desc="Testing")
for batch in progress_test:
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    # 填充模型的预测和标签后才能调用 gathere()
    predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
    predictions_gathered = accelerator.gather(predictions)
    labels_gathered = accelerator.gather(labels)
    true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
    # 添加到列表中
    all_predictions.extend(true_predictions)
    all_labels.extend(true_labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)
results = metric.compute()
print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

# 5. 保存预测结果到本地文件
import json
with open("test_predictions.json", "w") as f:
    json.dump(all_predictions, f)


In [ ]:
# 通过 token_classifier pipeline 测试模型
from transformers import pipeline

model = AutoModelForTokenClassification.from_pretrained(
    "model/bert-finetuned-ner-accelerate-epoch2"
)
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
token_classifier = pipeline(
    "token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

## 微调掩码语言模型（masked language model）


In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)